# Chapter 18: Variational Quantum Linear Solver

Variational Quantum Linear Solver (VQLS) and parametric circuits.

---

**Prerequisites:**
- See `Chapter02_QuantumSoftware.ipynb` for installation instructions


In [124]:
# Setup and imports
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import diags
from qiskit_aer import Aer
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import real_amplitudes as RealAmplitudes
from qiskit.quantum_info import SparsePauliOp, Statevector,Operator
from qiskit.primitives import StatevectorEstimator
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2
from scipy.optimize import minimize
from qiskit_algorithms.optimizers import SPSA

from Chapter08_QuantumGates_functions import (simulate_statevector, simulate_measurements, runCircuitOnIBMQuantum, 
                                              analyzeCircuitForHardware, plot_measurement_results)
print('Setup complete!')

Setup complete!


## Choose from Ax = b Examples

In [196]:
example = 2
if (example == 1):
	A = np.array([[1,0],[0,1]]) 
	b = np.array([1,0])
elif (example == 2):
	A = np.array([[2,-1],[-1,2]])
	b = np.array([1,1])/np.sqrt(2)
elif (example == 3):
	A = np.array([[1,0,0,-0.5],[0,1,0,0],[0,0,1,0],[-0.5,0,0,1]])
	b = np.array([1,0,0,0])
elif (example == 4):
	A = np.array([[1.5,0.5],[0.5,1.5]])
	b = np.array([1,0])
elif (example == 5):
	p = 1
	A = np.array([[5*(10**p),-1],[-1,5]])
	b = np.array([1,0])
elif (example == 6):
	N = 8
	values = [-np.ones(N-1),2*np.ones(N),-np.ones(N-1)]
	A = diags(values,[-1,0,1]).toarray()
	b = np.zeros(N)
	b[0] = 1

print("A:\n", A)
print("b:\n", b)

A:
 [[ 2 -1]
 [-1  2]]
b:
 [0.70710678 0.70710678]


## VQLS using Norm Minimization and StateVector

In [197]:
pauliSplit = SparsePauliOp.from_operator(A)
num_qubits = int(np.log2(A.shape[0]))

# 2. Setup the Ansatz
ansatz = RealAmplitudes(num_qubits, reps=6, entanglement='full' )
estimator = StatevectorEstimator()

# 3. Define the Cost Function (Global Norm)
def overlap_cost(params):
    bound_ansatz = ansatz.assign_parameters(params)
    state = Statevector(bound_ansatz)
    
    # Normalize A|u> so we only compare directions
    A_u = state.evolve(pauliSplit).data
    A_u_norm = A_u / np.linalg.norm(A_u)
    
    # Normalized b
    b_norm = b / np.linalg.norm(b)
    
    # Cost = 1 - |<b_norm | A_u_norm>|^2
    overlap = np.vdot(b_norm, A_u_norm)
    cost = 1 - np.abs(overlap)**2
    return float(cost)

# 4. Optimize
initial_theta = np.random.rand(ansatz.num_parameters)
#res = spsa.minimize(overlap_cost, initial_theta)
res = minimize(overlap_cost, initial_theta, method='L-BFGS-B', 
               options={'ftol': 1e-9, 'maxiter': 1000})
# --- 1. Get the Quantum Solution ---
final_circuit = ansatz.assign_parameters(res.x)
quantum_solution = Statevector(final_circuit).data

# Solve Ax = b classically
exact_solution = np.linalg.solve(A, b)
# Normalize the exact solution to unit length for comparison
exact_solution_norm = exact_solution / np.linalg.norm(exact_solution)

print("A:\n", A)
print("b:\n", b)
# Calculate Fidelity (how close the two vectors are)
print("Exact Solution (Normalized):", exact_solution_norm)
print("Quantum Solution:", quantum_solution)
fidelity = np.abs(np.vdot(exact_solution_norm, quantum_solution))**2
print(f"Solution Fidelity: {fidelity:.4f}")

A:
 [[ 2 -1]
 [-1  2]]
b:
 [0.70710678 0.70710678]
Exact Solution (Normalized): [0.70710678 0.70710678]
Quantum Solution: [0.70710678+0.j 0.70710678+0.j]
Solution Fidelity: 1.0000


## VQLS: Local vs Global with shots

In [198]:

nShots = 10000
use_global_cost = False
exact_estimation = True
nReps = 5
nGlobalIterations = 500

pauliSplit = SparsePauliOp.from_operator(A)
num_qubits = int(np.log2(A.shape[0]))
ansatz = RealAmplitudes(num_qubits, reps=nReps, entanglement='full' ) 
if (exact_estimation):
    estimator = StatevectorEstimator()  
else:
    # Create backend and estimator ONCE outside the cost function
    backend = AerSimulator()
    estimator = EstimatorV2(backend)
    estimator.options.default_shots = nShots

b_vec = np.array(b).flatten() 
b_normalized = b_vec / np.linalg.norm(b_vec)

# Now np.outer will correctly produce a 2D matrix (8x8 for your case)
b_density = np.outer(b_normalized, b_normalized.conj())
b_proj = SparsePauliOp.from_operator(b_density)
# Create the projector |b><b| and ensure it's Hermitian
b_density = np.outer(b_normalized, b_normalized.conj())
b_density = (b_density + b_density.conj().T) / 2  # Force Hermitian
b_observable = SparsePauliOp.from_operator(b_density).simplify()

# Force real coefficients (drop tiny imaginary parts from numerical error)
real_coeffs = np.real(b_observable.coeffs)
b_observable = SparsePauliOp(b_observable.paulis, real_coeffs)

def cost_func_shots_local(params):
    bound_ansatz = ansatz.assign_parameters(params)
    
    # 1. Term 1: <u| A†A |u> (Always Hermitian)
    a_dag_a = pauliSplit.adjoint().compose(pauliSplit).simplify()
    res1 = estimator.run([(bound_ansatz, a_dag_a)]).result()
    term1 = np.real(res1[0].data.evs)
    
    # 2. Term 2: 2 * Re(<b| A |u>)
    # Instead of one non-Hermitian op, we use the Hermitian 'Real part' operator
    O = b_proj.compose(pauliSplit)
    # Re(O) = 0.5 * (O + O_dagger)
    overlap_hermitian = (O + O.adjoint()).simplify() / 2
    
    res2 = estimator.run([(bound_ansatz, overlap_hermitian)]).result()
    # This expectation value now directly gives you Re(<b|A|u>)
    term2 = np.real(res2[0].data.evs)
    
    term3 = 1.0 
    
    # Cost = <u|A†A|u> - 2*Re(<b|A|u>) + 1
    cost = term1 - 2*term2 + term3
    return float(cost) if np.isscalar(cost) else cost.flatten().astype(float)

def cost_func_shots_global(params):
    bound_ansatz = ansatz.assign_parameters(params)
    
    # Compute A†A expectation
    a_dag_a = pauliSplit.adjoint().compose(pauliSplit).simplify()
    pub1 = (bound_ansatz, a_dag_a)
    result1 = estimator.run([pub1]).result()
    norm_val = result1[0].data.evs
    
    # Compute <b|A|u> for b = |0>
    pub2 = (bound_ansatz, pauliSplit)
    result2 = estimator.run([pub2]).result()
    overlap = result2[0].data.evs
    
    cost = 1 - (overlap**2 / norm_val)
    return float(cost)

# Optimize
initial_theta = np.random.rand(ansatz.num_parameters)
initial_theta = np.random.normal(0, 0.05, ansatz.num_parameters)
spsa = SPSA(
    maxiter=nGlobalIterations, 
    learning_rate=0.01, 
    perturbation=0.1
)
if use_global_cost:
    res = spsa.minimize(cost_func_shots_global, initial_theta)
else:
    res = spsa.minimize(cost_func_shots_local, initial_theta)

print(f"Final Cost: {res.fun}")

# Get solution
final_circuit = ansatz.assign_parameters(res.x)
quantum_solution = Statevector(final_circuit).data

exact_solution = np.linalg.solve(A, b)
exact_solution_norm = exact_solution / np.linalg.norm(exact_solution)

print("A:\n", A)
print("b:\n", b)
print("Exact Solution (Normalized):", exact_solution_norm)
print("Quantum Solution:", quantum_solution)
fidelity = np.abs(np.vdot(exact_solution_norm, quantum_solution))**2
print(f"Solution Fidelity: {fidelity:.4f}")

b_density shape: (2, 2)
b_proj shape: (2, 2)
Final Cost: -1.7763568394002505e-15
A:
 [[ 2 -1]
 [-1  2]]
b:
 [0.70710678 0.70710678]
Exact Solution (Normalized): [0.70710678 0.70710678]
Quantum Solution: [0.70710678+0.j 0.70710678+0.j]
Solution Fidelity: 1.0000
